In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [ ]:
df = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")
df.head()

In [ ]:
list = []
for i in df["ChEMBL ID"]:
    molecule = new_client.molecule
    m1 = molecule.filter(chembl_id=i).only(
        ['molecule_chembl_id', 'pref_name', 'molecule_structures'])
    targets = pd.DataFrame.from_dict(m1)
    #print(targets['molecule_chembl_id'][0])
    if targets['molecule_chembl_id'][0] == i:
        list.append([i,targets['pref_name'][0]])
        print(i,targets['pref_name'][0])
finals_2 = pd.DataFrame(list)

In [ ]:
finals_2